# Mark Galperin - Final Project
# Due December 8, 2020

This is my python notebook for the final project for ME 314! Wahoo

**Imports**

In [1]:
import sympy as sym 
from sympy.abc import t
print(sym.__version__)

1.6.2


In [2]:
#other imports...
from sympy import Function, Matrix, Eq, symbols
import numpy as np
import matplotlib.pyplot as plt

**Helper Functions**

In [3]:
def get_SE3_planar(R_th,p):
    """
    This function assembles planar homogenous transforms in SE(3) given either a rotation and a translation,
    or an R matrix and a translation.
    
    Takes...
        R_th: This input can be either a sympy matrix (at least 2x2) that is the rotation matrix for
            planar rotation about z, OR it can be a numerical value for the angle theta rotated about z
        
        p: a 3x1 (or 2x1) sympy matrix that is a homogenous coordinate vector for an x, y translation.
            p=0 is an acceptable input, shorthand for p = Matrix([0,0,0])
        
    Returns...
        g_SE3: a 3x3 sympy matrix that is a planar homogenous transform in SE(3)
    """
    
    #allowing p=0 as an input...
    if p==0:
        p = Matrix([0,0,0])
    
    #assembling the matrix...
    if type(R_th).__name__ == 'MutableDenseMatrix':
        #given R_th is a matrix in SE(2) at least... 
        g_SE3 = Matrix([[R_th[0,0], R_th[0,1], 0, p[0]],
                  [R_th[1,0], R_th[1,1], 0, p[1]],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])
    else:   
        #if R_or_th is a numerical value...
        g_SE3 = Matrix([[sym.cos(R_th), -sym.sin(R_th), 0, p[0]],
                      [sym.sin(R_th), sym.cos(R_th), 0, p[1]],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])
    
    return g_SE3

In [4]:
def get_SE3_planar_np(R_th,p):
    """
    This function assembles planar homogenous transforms in SE(3) given either a rotation and a translation,
    or an R matrix and a translation.
    
    Takes...
        R_th: This input can be either a sympy matrix (at least 2x2) that is the rotation matrix for
            planar rotation about z, OR it can be a numerical value for the angle theta rotated about z
        
        p: a 3x1 (or 2x1) sympy matrix that is a homogenous coordinate vector for an x, y translation.
            p=0 is NOT an acceptable shorthand here. use p = np.array([0,0,0])
        
    Returns...
        g_SE3: a 3x3 numpy array that is a planar homogenous transform in SE(3)
    """
    
    #assembling the matrix...
    if type(R_th).__name__ == 'ndarray':
        #given R_th is a matrix in SE(2) at least... 
        g_SE3 = np.array([[R_th[0][0], R_th[0][1], 0, p[0]],
                  [R_th[1][0], R_th[1][1], 0, p[1]],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])
    else:   
        #if R_or_th is a numerical value...
        g_SE3 = np.array([[np.cos(R_th), -np.sin(R_th), 0, p[0]],
                      [np.sin(R_th), np.cos(R_th), 0, p[1]],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])
    
    return g_SE3

In [5]:
thnp = np.pi/3
pnp = np.array([1,2,3])
get_SE3_planar_np(thnp,pnp)

array([[ 0.5      , -0.8660254,  0.       ,  1.       ],
       [ 0.8660254,  0.5      ,  0.       ,  2.       ],
       [ 0.       ,  0.       ,  1.       ,  0.       ],
       [ 0.       ,  0.       ,  0.       ,  1.       ]])

In [6]:
def hat(w):
    w_hat = Matrix([[0,    -w[2], w[1] ],
                    [w[2],   0,  -w[0] ],
                    [-w[1], w[0],   0  ]])
    return w_hat

In [7]:
def unhat(w_hat):
    return Matrix([w_hat[2,1],w_hat[0,2],w_hat[1,0]])

In [8]:
def g_inv(g):
    """
    This function calculates the inverse of a transformation g, in (planar) SE(3). 
    
    Dependencies:
        get_SE3_planar(th,p): Generates a planar homogenous transform in SE(3). Used for output.
    
    Takes...
        g: a 3x3 sympy matrix in SE(3)
        
    Returns...
        g_out: a 3x3 sympy matrix in SE(3) that is the "inverse" of g
    """
    #dissecting the input transformation g
    R = g[0:3,0:3]
    p = Matrix([g[0,3],g[1,3],1])
    
    RT = R.T
    p_inv = -RT*p
    
    return get_SE3_planar(RT,p_inv)

In [9]:
def g_dot(g,q_th):
    """
    This function gives the time-derivative of g
    g_dot = [R_dot p_dot
             0     1    ]
    ...where R_dot = Rw = wR, and w = [0 0 d(th)/dt]
    
    TODO - more docstring
    """
    
    w = Matrix([0,0,q_th.diff(t)])
    
    #R_dot...
    R = g[0:3,0:3]
    R_dot = R*hat(w)
    
    #p_dot...
    p = g[0:3,3]
    p_dot = p.diff(t)
    
    #generating the transform...
    return get_SE3_planar(R_dot,p_dot)

In [10]:
def get_Vb(g,q_th,select='Vb'):
    
    #TODO: selection dictionary
    
    Vhat = g_inv(g)*g_dot(g,q_th)
    w_hat = Vhat[0:3,0:3]
    v = Vhat[0:3,3]
    w = unhat(w_hat)
    Vb = v.col_join(w)
    return Vb

In [11]:
a = np.array([[1,2,3],[4,5,6],[7,8,9]])
type(a).__name__

'ndarray'

## Input Variables

In [12]:
#Input Parameters...
R = 1 #the radius of the inscribed circle for the n-gon
n = 7 #number of sides for the n-gon. n >= 3. 
N = 3 #number of masses (PREFERABLY LESS THAN 4)
r = 0.05 #radius of the masses

#Geometric Parameters
W = 0.01 #wall thickness
L = R*np.tan(np.pi/n)/2 #wall length (equal to side length of the n-gon)

## Setup

**Symbols, and Configuration**

In [13]:
# other symbols...
grav = 9.81

# Time functions...
th = Function('θ')(t)

#initializing the configuration list...
q_list = [th]

# Generating configuration variables from N...
for i in range(N):
    qix = Function(f'x_{i+1}')(t)
    qiy = Function(f'y_{i+1}')(t)
    q_list.append(qix)
    q_list.append(qiy)

#config vector and derivatives
q = Matrix(q_list)
qdot = q.diff(t)
qddot = qdot.diff(t)


#Display
print('The configuration vector:')
display(q)

#Warning
if len(q) > 7:
    print('WARNING!! THIS IS TOO MUCH!!')

The configuration vector:


Matrix([
[  θ(t)],
[x_1(t)],
[y_1(t)],
[x_2(t)],
[y_2(t)],
[x_3(t)],
[y_3(t)]])

**Defining Frames for the n-gon**
Due to the parametric nature of this set-up, the number of frames will be dependent on n, the number of sides of the polygonal tumbler.

In [14]:
#Defining a dict g of transformations...
g = {}
g_sym = {}
angdiv = 2*np.pi/n

for i in range(n):
    #rotate by theta, plus the fraction of 2pi, plus pi/2 to make it tangent
    ang = q[0] + i*angdiv + np.pi/2
    g_R = get_SE3_planar(ang,0)
    #translate in the new frame by -R
    p = Matrix([0,-R])
    g_p = get_SE3_planar(0,p)
    
    #Add to dictionary
    g[f'w{i+1}'] = g_R*g_p
    g_sym[f'w{i+1}'] = symbols(f'g_w{i+1}')

    
#display...
print(f'These are the transformations for the sides of a {n}-sided tumbler')
for tfm in g.keys():
    display(Eq(g_sym[tfm],g[tfm],evaluate=0))

These are the transformations for the sides of a 7-sided tumbler


Eq(g_w1, Matrix([
[cos(θ(t) + 1.5707963267949), -sin(θ(t) + 1.5707963267949), 0,  sin(θ(t) + 1.5707963267949)],
[sin(θ(t) + 1.5707963267949),  cos(θ(t) + 1.5707963267949), 0, -cos(θ(t) + 1.5707963267949)],
[                          0,                            0, 1,                            0],
[                          0,                            0, 0,                            1]]))

Eq(g_w2, Matrix([
[cos(θ(t) + 2.46839422782055), -sin(θ(t) + 2.46839422782055), 0,  sin(θ(t) + 2.46839422782055)],
[sin(θ(t) + 2.46839422782055),  cos(θ(t) + 2.46839422782055), 0, -cos(θ(t) + 2.46839422782055)],
[                           0,                             0, 1,                             0],
[                           0,                             0, 0,                             1]]))

Eq(g_w3, Matrix([
[cos(θ(t) + 3.36599212884621), -sin(θ(t) + 3.36599212884621), 0,  sin(θ(t) + 3.36599212884621)],
[sin(θ(t) + 3.36599212884621),  cos(θ(t) + 3.36599212884621), 0, -cos(θ(t) + 3.36599212884621)],
[                           0,                             0, 1,                             0],
[                           0,                             0, 0,                             1]]))

Eq(g_w4, Matrix([
[cos(θ(t) + 4.26359002987186), -sin(θ(t) + 4.26359002987186), 0,  sin(θ(t) + 4.26359002987186)],
[sin(θ(t) + 4.26359002987186),  cos(θ(t) + 4.26359002987186), 0, -cos(θ(t) + 4.26359002987186)],
[                           0,                             0, 1,                             0],
[                           0,                             0, 0,                             1]]))

Eq(g_w5, Matrix([
[cos(θ(t) + 5.16118793089752), -sin(θ(t) + 5.16118793089752), 0,  sin(θ(t) + 5.16118793089752)],
[sin(θ(t) + 5.16118793089752),  cos(θ(t) + 5.16118793089752), 0, -cos(θ(t) + 5.16118793089752)],
[                           0,                             0, 1,                             0],
[                           0,                             0, 0,                             1]]))

Eq(g_w6, Matrix([
[cos(θ(t) + 6.05878583192317), -sin(θ(t) + 6.05878583192317), 0,  sin(θ(t) + 6.05878583192317)],
[sin(θ(t) + 6.05878583192317),  cos(θ(t) + 6.05878583192317), 0, -cos(θ(t) + 6.05878583192317)],
[                           0,                             0, 1,                             0],
[                           0,                             0, 0,                             1]]))

Eq(g_w7, Matrix([
[cos(θ(t) + 6.95638373294883), -sin(θ(t) + 6.95638373294883), 0,  sin(θ(t) + 6.95638373294883)],
[sin(θ(t) + 6.95638373294883),  cos(θ(t) + 6.95638373294883), 0, -cos(θ(t) + 6.95638373294883)],
[                           0,                             0, 1,                             0],
[                           0,                             0, 0,                             1]]))

In [15]:
# from IPython.core.display import HTML
# display(HTML("<table><tr><td><img src='https://github.com/MarkGalperin/Mechanical-Engineering-314/blob/main/Images/pizza%20cat.jpg?raw=true' width=700' height='350'></table>"))

**Inertial Properties: Tumbler** <br> 
As it stands, I'm defining each wall of the tumbler to have identical inertial properties

In [16]:
#Inertial parameters for each wall... TODO-make more geometry-based?
m_t = 1
J_t = 1

In [17]:
#defining I for n sides...
I = {}
for i in range(n):
    Imat = Matrix([[m_t,0,0,0,0,0],
                    [0,m_t,0,0,0,0],
                    [0,0,m_t,0,0,0],
                    [0,0,0,0,0,0],
                    [0,0,0,0,0,0],
                    [0,0,0,0,0,J_t]])
    #Add to dictionary
    I[f'w{i+1}'] = Imat

**Body Velocity: Tumbler**

In [18]:
#Getting the V for each frame...
V = {}
V_sym = {}

print("The body velocities for each frame:")
for frame in g:
    #body velocity and display symbol...
    Vb = get_Vb(g[frame],q[0])
    Vsym = symbols(f'V_{frame[1:]}')
    #add to dictionaries...
    V[frame] = Vb
    V_sym[frame] = Vsym
    #display...
    display(Eq(Vsym,Vb,evaluate=0))

The body velocities for each frame:


Eq(V_1, Matrix([
[sin(θ(t) + 1.5707963267949)**2*Derivative(θ(t), t) + cos(θ(t) + 1.5707963267949)**2*Derivative(θ(t), t)],
[                                        sin(θ(t) + 1.5707963267949)**2 + cos(θ(t) + 1.5707963267949)**2],
[                                                                                                      0],
[                                                                                                      0],
[                                                                                                      0],
[sin(θ(t) + 1.5707963267949)**2*Derivative(θ(t), t) + cos(θ(t) + 1.5707963267949)**2*Derivative(θ(t), t)]]))

Eq(V_2, Matrix([
[sin(θ(t) + 2.46839422782055)**2*Derivative(θ(t), t) + cos(θ(t) + 2.46839422782055)**2*Derivative(θ(t), t)],
[                                        sin(θ(t) + 2.46839422782055)**2 + cos(θ(t) + 2.46839422782055)**2],
[                                                                                                        0],
[                                                                                                        0],
[                                                                                                        0],
[sin(θ(t) + 2.46839422782055)**2*Derivative(θ(t), t) + cos(θ(t) + 2.46839422782055)**2*Derivative(θ(t), t)]]))

Eq(V_3, Matrix([
[sin(θ(t) + 3.36599212884621)**2*Derivative(θ(t), t) + cos(θ(t) + 3.36599212884621)**2*Derivative(θ(t), t)],
[                                        sin(θ(t) + 3.36599212884621)**2 + cos(θ(t) + 3.36599212884621)**2],
[                                                                                                        0],
[                                                                                                        0],
[                                                                                                        0],
[sin(θ(t) + 3.36599212884621)**2*Derivative(θ(t), t) + cos(θ(t) + 3.36599212884621)**2*Derivative(θ(t), t)]]))

Eq(V_4, Matrix([
[sin(θ(t) + 4.26359002987186)**2*Derivative(θ(t), t) + cos(θ(t) + 4.26359002987186)**2*Derivative(θ(t), t)],
[                                        sin(θ(t) + 4.26359002987186)**2 + cos(θ(t) + 4.26359002987186)**2],
[                                                                                                        0],
[                                                                                                        0],
[                                                                                                        0],
[sin(θ(t) + 4.26359002987186)**2*Derivative(θ(t), t) + cos(θ(t) + 4.26359002987186)**2*Derivative(θ(t), t)]]))

Eq(V_5, Matrix([
[sin(θ(t) + 5.16118793089752)**2*Derivative(θ(t), t) + cos(θ(t) + 5.16118793089752)**2*Derivative(θ(t), t)],
[                                        sin(θ(t) + 5.16118793089752)**2 + cos(θ(t) + 5.16118793089752)**2],
[                                                                                                        0],
[                                                                                                        0],
[                                                                                                        0],
[sin(θ(t) + 5.16118793089752)**2*Derivative(θ(t), t) + cos(θ(t) + 5.16118793089752)**2*Derivative(θ(t), t)]]))

Eq(V_6, Matrix([
[sin(θ(t) + 6.05878583192317)**2*Derivative(θ(t), t) + cos(θ(t) + 6.05878583192317)**2*Derivative(θ(t), t)],
[                                        sin(θ(t) + 6.05878583192317)**2 + cos(θ(t) + 6.05878583192317)**2],
[                                                                                                        0],
[                                                                                                        0],
[                                                                                                        0],
[sin(θ(t) + 6.05878583192317)**2*Derivative(θ(t), t) + cos(θ(t) + 6.05878583192317)**2*Derivative(θ(t), t)]]))

Eq(V_7, Matrix([
[sin(θ(t) + 6.95638373294883)**2*Derivative(θ(t), t) + cos(θ(t) + 6.95638373294883)**2*Derivative(θ(t), t)],
[                                        sin(θ(t) + 6.95638373294883)**2 + cos(θ(t) + 6.95638373294883)**2],
[                                                                                                        0],
[                                                                                                        0],
[                                                                                                        0],
[sin(θ(t) + 6.95638373294883)**2*Derivative(θ(t), t) + cos(θ(t) + 6.95638373294883)**2*Derivative(θ(t), t)]]))

**Energy: Tumbler**

Kinetic...

In [19]:
#KE for each frame...
KEdict = {}
KE_sym = {}
for frame in g:
    KEdict[frame] = ((1/2)*V[frame].T*I[frame]*V[frame])[0]
    KE_sym[frame] = symbols(f'KE_{frame[1:]}')
    
#summing up the total KE...
KE = sym.simplify(sum(KEdict.values()))

print(f'Kinetic energy (from the {n}-sided tumbler)')
display(KE)

Kinetic energy (from the 7-sided tumbler)


7.0*Derivative(θ(t), t)**2 + 3.5

Potential...

In [29]:
PEdict = {}
PE_sym = {}
for frame in g:
    #y coordinate in the world frame
    y_w = g[frame][1,3] #2nd row, 4th column of SE(3)
    PEdict[frame] = (m_t*grav*y_w)

PE = sym.simplify(sum(PEdict.values()))

print(f'Potential energy from the {n}-sided tumbler...')
print('note: this is essentially zero')
display(PE)

Potential energy from the 7-sided tumbler...
note: this is essentially zero


-3.26738636147184e-15*sin(θ(t)) - 1.08912878715728e-15*cos(θ(t))

In [31]:
L = KE-PE
display(L)

3.26738636147184e-15*sin(θ(t)) + 1.08912878715728e-15*cos(θ(t)) + 7.0*Derivative(θ(t), t)**2 + 3.5

In [23]:
PEdict['w1']

-9.81*cos(θ(t) + 1.5707963267949)

## Animation

In [21]:
dontruntheseceeellss

NameError: name 'dontruntheseceeellss' is not defined

In [ ]:
#making dummy data for a plot...
ls = np.linspace(0,10,100)
faketraj = np.vstack((ls,ls))

In [ ]:
def animate_tumbler(q_array,T=10):
    """
    Function to generate web-based animation of just the raffle tumbler

    Parameters:
    ================================================
    q_array:
        trajectory of theta1 and theta2, should be a NumPy array with
        shape of (2,N)
    T:
        length/seconds of animation duration

    Returns: None
    """

    ################################
    # Imports required for animation.
    from plotly.offline import init_notebook_mode, iplot
    from IPython.display import display, HTML
    import plotly.graph_objects as go

    #######################
    # Browser configuration.
    def configure_plotly_browser_state():
        import IPython
        display(IPython.core.display.HTML('''
            <script src="/static/components/requirejs/require.js"></script>
            <script>
              requirejs.config({
                paths: {
                  base: '/static/base',
                  plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
                },
              });
            </script>
            '''))
    configure_plotly_browser_state()
    init_notebook_mode(connected=False)

    ###############################################
    # Length of trajectory
    N = len(q_array[0])

    ###############################################
    # Define arrays containing data for frame axes
    # In each frame, the x and y axis are always fixed
    frame_length = 0.1
    x_axis = np.array([frame_length, 0.0])
    y_axis = np.array([0.0, frame_length])
    origin = np.array([0.0, 0.0])
    
    #rectangle
#     rec1_bl = np.array([-W1/2., 0.0])
#     rec1_br = np.array([W1/2., 0.0])
#     rec1_tl = np.array([-W1/2., L1])
#     rec1_tr = np.array([W1/2., L1])
    
#     rec2_bl = np.array([-W2/2., 0.0])
#     rec2_br = np.array([W2/2., 0.0])
#     rec2_tl = np.array([-W2/2., L2])
#     rec2_tr = np.array([W2/2., L2])
    
    # Initialize an array for each point
#     frame_a_x_axis = np.zeros((2,N))
#     frame_a_y_axis = np.zeros((2,N))
#     frame_a_origin = np.zeros((2,N))
    
#     rec1_bl_w = np.zeros((2,N))
#     rec1_br_w = np.zeros((2,N))
#     rec1_tl_w = np.zeros((2,N))
#     rec1_tr_w = np.zeros((2,N))
    
    #frame dictionary
    framedict = {}
    for ii in range(n):
        #initialize np arrays for origin,x,y
        origin_array = np.zeros((2,N))
        x_axis_array = np.zeros((2,N))
        y_axis_array = np.zeros((2,N))
        
        #Add to dictionary. This dictionary will store all point data for each frame, and concise my code!!! waahoooo
        framedict[f'w{ii+1}'] = [origin_array,x_axis_array,y_axis_array]
    
    for i in range(N): # iteration through each time step
        
        #transformation dictionary...
        g_np = {}
        angdiv = 2*np.pi/n
        for ii in range(n):
            #rotate by theta, plus the fraction of 2pi, plus pi/2 to make it tangent
            ang = q_array[0][i] + ii*angdiv + np.pi/2
            g_R = get_SE3_planar_np(ang,np.array([0,0,0]))
            #translate in the new frame by -R
            p = np.array([0,-R,0])
            g_p = get_SE3_planar_np(0,p)

            #Add to dictionary
            g_np[f'w{ii+1}'] = np.matmul(g_R,g_p)
    
        
        # transfer the x axis, y axis, and origins in body frame back to world frame at 
        # the current time step. Overwrites the zero values in the dictionary.
#         print(f'i is {i}, theta is {q_array[0][i]}')
        for tfm in framedict:
#             print(f'key={tfm}, type={type(tfm)}')
#             print(framedict[tfm][0][:,i])
            framedict[tfm][0][:,i] = g_np[tfm].dot([origin[0], origin[1], 0, 1])[0:2] #origin
            framedict[tfm][1][:,i] = g_np[tfm].dot([x_axis[0], x_axis[1], 0, 1])[0:2] #x
            framedict[tfm][2][:,i] = g_np[tfm].dot([y_axis[0], y_axis[1], 0, 1])[0:2] #y
        
        #Setting up the rectangle
#         rec1_bl_w[:,i] = t_wd.dot([rec1_bl[0], rec1_bl[1], 1])[0:2]
#         rec1_br_w[:,i] = t_wd.dot([rec1_br[0], rec1_br[1], 1])[0:2]
#         rec1_tl_w[:,i] = t_wd.dot([rec1_tl[0], rec1_tl[1], 1])[0:2]
#         rec1_tr_w[:,i] = t_wd.dot([rec1_tr[0], rec1_tr[1], 1])[0:2]
        
#         rec2_bl_w[:,i] = t_we.dot([rec2_bl[0], rec2_bl[1], 1])[0:2]
#         rec2_br_w[:,i] = t_we.dot([rec2_br[0], rec2_br[1], 1])[0:2]
#         rec2_tl_w[:,i] = t_we.dot([rec2_tl[0], rec2_tl[1], 1])[0:2]
#         rec2_tr_w[:,i] = t_we.dot([rec2_tr[0], rec2_tr[1], 1])[0:2]
    
    ####################################
    # Using these to specify axis limits.
    xm = -1.5 #np.min(xx1)-0.5
    xM = 1.5 #np.max(xx1)+0.5
    ym = -1.5 #np.min(yy1)-2.5
    yM = 1.5 #np.max(yy1)+1.5

    ###########################
    # Defining data dictionary.
    # Trajectories are here.
    
    #inserting generated 'data' dictionaries...
    gendata = []
    for tfm in framedict.keys():
        x_label = dict(name=f'{tfm} frame x')
        y_label = dict(name=f'{tfm} frame y')
        gendata.append(x_label)
        gendata.append(y_label)
    
    data=[
        # note that except for the trajectory (which you don't need this time),
        # you don't need to define entries other than "name". The items defined
        # in this list will be related to the items defined in the "frames" list
        # later in the same order. Therefore, these entries can be considered as 
        # labels for the components in each animation frame
        dict(name='w frame X'),
        dict(name='w frame Y'),
        *gendata,

        # You don't need to show trajectory this time,
        # but if you want to show the whole trajectory in the animation (like what
        # you did in previous homeworks), you will need to define entries other than 
        # "name", such as "x", "y". and "mode".

        # dict(x=xx1, y=yy1, 
        #      mode='markers', name='Pendulum 1 Traj', 
        #      marker=dict(color="fuchsia", size=2)
        #     ),
        # dict(x=xx2, y=yy2, 
        #      mode='markers', name='Pendulum 2 Traj', 
        #      marker=dict(color="purple", size=2)
        #     ),
        ]

    ################################
    # Preparing simulation layout.
    # Title and axis ranges are here.
    layout=dict(autosize=False, width=1000, height=1000,
                xaxis=dict(range=[xm, xM], autorange=False, zeroline=True,dtick=1),
                yaxis=dict(range=[ym, yM], autorange=False, zeroline=False,scaleanchor = "x",dtick=1),
                title='Double Pendulum Simulation', 
                hovermode='closest',
                updatemenus= [{'type': 'buttons',
                               'buttons': [{'label': 'Play','method': 'animate',
                                            'args': [None, {'frame': {'duration': T, 'redraw': False}}]},
                                           {'args': [[None], {'frame': {'duration': T, 'redraw': False}, 'mode': 'immediate',
                                            'transition': {'duration': 0}}],'label': 'Pause','method': 'animate'}
                                          ]
                              }]
               )

    ########################################
    # Defining the frames of the simulation.
    
    #I took it out of the list comprehension to include generated data. And also list comprehensions make my brain hurt!
    frames = []
    for k in range(N):
        #generated frame data...
        gendata = []
        for tfm in framedict.keys():
            #draw out the x and y lines...
            x_line = dict(x=[framedict[tfm][0][0][k], framedict[tfm][1][0][k]], 
                         y=[framedict[tfm][0][1][k], framedict[tfm][1][1][k]], 
                         mode='lines',
                         line=dict(color='orange', width=3),
                        )
            y_line = dict(x=[framedict[tfm][0][0][k], framedict[tfm][2][0][k]], 
                         y=[framedict[tfm][0][1][k], framedict[tfm][2][1][k]], 
                         mode='lines',
                         line=dict(color='orange', width=3),
                        )
            #add to gendata...
            gendata.append(x_line)
            gendata.append(y_line)
        
        frames.append(dict(data=[# first three objects correspond to the arms and two masses,
                       # same order as in the "data" variable defined above (thus 
                       # they will be labeled in the same order)

                       # display x and y axes of the fixed frame in each animation frame
                       dict(x=[0,x_axis[0]], 
                            y=[0,x_axis[1]], 
                            mode='lines',
                            line=dict(color='red', width=3),
                            ),
                       dict(x=[0,y_axis[0]], 
                            y=[0,y_axis[1]], 
                            mode='lines',
                            line=dict(color='red', width=3),
                            ),
        
                        #unpacking the generated data...
                        *gendata,
#                     #RECTANGLE1
#                        dict(x=[rec1_bl_w[0][k], rec1_br_w[0][k]], 
#                             y=[rec1_bl_w[1][k], rec1_br_w[1][k]],
#                             mode='lines',
#                             line=dict(color='red', width=3),
#                             ),
#                        dict(x=[rec1_br_w[0][k], rec1_tr_w[0][k]], 
#                             y=[rec1_br_w[1][k], rec1_tr_w[1][k]],
#                             mode='lines',
#                             line=dict(color='red', width=3),
#                             ),
#                        dict(x=[rec1_tl_w[0][k], rec1_tr_w[0][k]], 
#                             y=[rec1_tl_w[1][k], rec1_tr_w[1][k]],
#                             mode='lines',
#                             line=dict(color='red', width=3),
#                             ),
#                        dict(x=[rec1_bl_w[0][k], rec1_tl_w[0][k]], 
#                             y=[rec1_bl_w[1][k], rec1_tl_w[1][k]],
#                             mode='lines',
#                             line=dict(color='red', width=3),
#                             ),
                      ]))
                    

    #######################################
    # Putting it all together and plotting.
    figure1=dict(data=data, layout=layout, frames=frames)           
    iplot(figure1)
    
    return framedict

In [ ]:
fd = animate_tumbler(faketraj,T=100)

In [ ]:
fd.keys()

In [ ]:
w1_dbg = fd['w1'][0]